In [1]:
import numpy as np
import pandas as pd

df = pd.read_csv('data/D_train.csv')
df.describe()

,Unnamed: 0,Class,User,X0,Y0,Z0,X1,Y1,Z1,X2,...,Z8,X9,Y9,Z9,X10,Y10,Z10,X11,Y11,Z11
count,13500.000000,13500.000000,13500.000000,13500.000000,13500.000000,13500.000000,13500.000000,13500.000000,13500.000000,13500.000000,...,5733.000000,4537.000000,4537.000000,4537.000000,2482.000000,2482.000000,2482.000000,0.0,0.0,0.0
mean,27296.628370,3.025481,5.777778,52.698144,85.770524,-31.826014,52.351641,86.272265,-31.711906,52.112129,...,-28.776171,57.177513,83.183279,-28.576656,53.669544,77.761579,-29.641994,NaN,NaN,NaN
std,15051.282199,1.445896,3.823397,32.676394,40.715829,33.898217,32.353705,40.641751,34.163914,33.570688,...,35.921888,41.614585,43.932179,37.058562,40.719084,46.033896,42.029159,NaN,NaN,NaN
min,5.000000,1.000000,0.000000,-80.928512,-98.233756,-120.096446,-111.685241,-96.142589,-114.271334,-106.886524,...,-114.500502,-83.649652,-39.539982,-112.908778,-80.196289,-2.877761,-103.718071,NaN,NaN,NaN
25%,14897.500000,2.000000,2.000000,31.260337,60.571185,-58.246757,30.917769,62.849522,-59.477546,29.413095,...,-59.907133,27.634371,44.872205,-58.906215,14.298679,37.354595,-70.016161,NaN,NaN,NaN
50%,25943.000000,3.000000,6.000000,55.336918,87.314804,-31.697639,55.415571,88.220022,-31.292537,56.038056,...,-27.741830,63.516259,87.992227,-27.794790,62.781569,84.955399,-31.392546,NaN,NaN,NaN
75%,38387.500000,4.000000,9.000000,75.288905,106.228314,-5.744637,75.038670,109.185672,-4.650574,75.422747,...,0.410206,86.090881,126.478059,1.340623,84.088250,121.913529,6.843351,NaN,NaN,NaN
max,56980.000000,5.000000,11.000000,151.586035,168.717458,113.345119,151.271413,170.209350,104.618115,149.208278,...,119.213101,174.054403,167.196644,122.569627,149.486224,168.352478,108.211488,NaN,NaN,NaN


In [2]:
from postures import GeometricFeatures
# df = GeometricFeatures().fit_transform(df)
df.describe()

,id,class,user,n_markers,x_mean,x_std,x_min,x_max,y_mean,y_std,...,ang_min,ang_max,area_mean,area_std,area_min,area_max,conv_hull_vol,conv_hull_cx,conv_hull_cy,conv_hull_cz
count,13500.000000,13500.000000,13500.000000,13500.000000,13500.000000,13500.000000,13500.000000,13500.000000,13500.000000,13500.000000,...,13500.000000,13500.000000,13500.000000,13500.000000,13500.000000,13500.000000,1.350000e+04,13500.000000,13500.000000,13500.000000
mean,27296.628370,3.025481,5.777778,7.826519,51.247539,30.123615,2.219060,93.288206,80.668318,36.758654,...,1.872420,3.024153,3623.656033,1827.232683,802.611350,7067.486140,1.822859e+05,45.216215,70.523444,-30.283510
std,15051.282199,1.445896,3.823397,1.917432,14.839614,7.922490,22.893672,20.733971,17.564579,11.190715,...,0.263571,0.055370,881.971581,622.010039,345.904671,2201.129666,1.531822e+05,13.015313,17.229727,17.776387
min,5.000000,1.000000,0.000000,3.000000,-61.898200,1.877600,-121.182100,-27.857600,-50.301800,2.545300,...,0.361600,2.323400,1608.559000,597.788700,14.763400,2548.911200,9.154100e+01,-51.581800,-41.918200,-77.306500
25%,14897.500000,2.000000,2.000000,6.000000,39.657825,25.134850,-12.995725,78.862675,67.838725,30.728650,...,1.708175,2.977200,2837.848675,1410.366400,554.179975,5785.377075,7.466991e+04,35.828450,58.274775,-42.543250
50%,25943.000000,3.000000,6.000000,8.000000,52.159600,31.152150,-2.771700,91.763700,81.522250,38.331250,...,1.841200,3.020200,3886.932650,1822.077800,832.972700,7310.103700,1.436213e+05,45.181350,71.055100,-32.173500
75%,38387.500000,4.000000,9.000000,10.000000,62.215525,34.327275,13.468450,108.600925,94.285750,44.068700,...,1.998125,3.069300,4260.251700,2285.970175,1023.244850,8732.409300,2.304890e+05,55.544100,84.634375,-17.190800
max,56980.000000,5.000000,11.000000,10.000000,102.433600,69.833700,68.288100,174.054400,136.535400,65.277300,...,2.797300,3.139900,7192.107400,3992.647000,3405.102400,16612.221600,1.435080e+06,88.143700,114.932800,68.334000


In [3]:
# Save datasets
df.to_csv('data/train_processed.csv', index = False)

In [5]:
df = pd.read_csv('data/train_processed.csv')
df.describe()

,id,class,user,n_markers,x_mean,x_std,x_min,x_max,y_mean,y_std,...,ang_min,ang_max,area_mean,area_std,area_min,area_max,conv_hull_vol,conv_hull_cx,conv_hull_cy,conv_hull_cz
count,13500.000000,13500.000000,13500.000000,13500.000000,13500.000000,13500.000000,13500.000000,13500.000000,13500.000000,13500.000000,...,13500.000000,13500.000000,13500.000000,13500.000000,13500.000000,13500.000000,1.350000e+04,13500.000000,13500.000000,13500.000000
mean,27296.628370,3.025481,5.777778,7.826519,51.247539,30.123615,2.219060,93.288206,80.668318,36.758654,...,1.872420,3.024153,3623.656033,1827.232683,802.611350,7067.486140,1.822859e+05,45.216215,70.523444,-30.283510
std,15051.282199,1.445896,3.823397,1.917432,14.839614,7.922490,22.893672,20.733971,17.564579,11.190715,...,0.263571,0.055370,881.971581,622.010039,345.904671,2201.129666,1.531822e+05,13.015313,17.229727,17.776387
min,5.000000,1.000000,0.000000,3.000000,-61.898200,1.877600,-121.182100,-27.857600,-50.301800,2.545300,...,0.361600,2.323400,1608.559000,597.788700,14.763400,2548.911200,9.154100e+01,-51.581800,-41.918200,-77.306500
25%,14897.500000,2.000000,2.000000,6.000000,39.657825,25.134850,-12.995725,78.862675,67.838725,30.728650,...,1.708175,2.977200,2837.848675,1410.366400,554.179975,5785.377075,7.466991e+04,35.828450,58.274775,-42.543250
50%,25943.000000,3.000000,6.000000,8.000000,52.159600,31.152150,-2.771700,91.763700,81.522250,38.331250,...,1.841200,3.020200,3886.932650,1822.077800,832.972700,7310.103700,1.436213e+05,45.181350,71.055100,-32.173500
75%,38387.500000,4.000000,9.000000,10.000000,62.215525,34.327275,13.468450,108.600925,94.285750,44.068700,...,1.998125,3.069300,4260.251700,2285.970175,1023.244850,8732.409300,2.304890e+05,55.544100,84.634375,-17.190800
max,56980.000000,5.000000,11.000000,10.000000,102.433600,69.833700,68.288100,174.054400,136.535400,65.277300,...,2.797300,3.139900,7192.107400,3992.647000,3405.102400,16612.221600,1.435080e+06,88.143700,114.932800,68.334000
